TEST EN TS AVEC LANGGRAPH D'UNE ORCHESTRATION DE MULTI AGENTS

In [1]:
import { END, Annotation } from "npm:@langchain/langgraph";
import { BaseMessage } from "npm:@langchain/core/messages";

// This defines the object that is passed between each node
// in the graph. We will create different nodes for each agent and tool
const AgentState = Annotation.Root({
    messages: Annotation<BaseMessage[]>({
        reducer: (x, y) => x.concat(y),
        default: () => [],
    }),
    // The agent node that last performed work
    next: Annotation<string>({
        reducer: (x, y) => y ?? x ?? END,
        default: () => END,
    }),
});

In [2]:
process.env.TAVILY_API_KEY = "";
process.env.MISTRAL_API_KEY = "";


""

In [3]:
// import "npm:tsx"; // Only for running this in TSLab. See: https://github.com/yunabe/tslab/issues/72
import { TavilySearchResults } from "npm:@langchain/community/tools/tavily_search";
import { DynamicStructuredTool } from "npm:@langchain/core/tools";
import * as d3 from "npm:d3";
// ----------ATTENTION----------
// If attempting to run this notebook locally, you must follow these instructions
// to install the necessary system dependencies for the `canvas` package.
// https://www.npmjs.com/package/canvas#compiling
// -----------------------------
// import { createCanvas } from "canvas";
// import { z } from "zod";
// import * as tslab from "tslab";

const additionerTool = new DynamicStructuredTool({
    name: "number_additionner",
    description: "Additionne deux nombres",
    func: (a: number, b: number) => { return a + b},
});

const tavilyTool = new TavilySearchResults();

In [4]:
import { z } from "npm:zod";
import { ChatMistralAI } from "npm:@langchain/mistralai";

import { ChatPromptTemplate, MessagesPlaceholder } from "npm:@langchain/core/prompts";

const members = ["researcher", "number_additionner"] as const;

const systemPrompt =
    "You are a supervisor tasked with managing a conversation between the" +
    " following workers: {members}. Given the following user request," +
    " respond with the worker to act next. Each worker will perform a" +
    " task and respond with their results and status. When finished," +
    " respond with FINISH.";
const options = [END, ...members];

// Define the routing function
const routingTool = {
    name: "route",
    description: "Select the next role.",
    schema: z.object({
        next: z.enum([END, ...members]),
    }),
}

const prompt = ChatPromptTemplate.fromMessages([
    ["system", systemPrompt],
    new MessagesPlaceholder("messages"),
    [
        "human",
        "Given the conversation above, who should act next?" +
        " Or should we FINISH? Select one of: {options}",
    ],
]);

const formattedPrompt = await prompt.partial({
    options: options.join(", "),
    members: members.join(", "),
});

const llm = new ChatMistralAI({
    modelName: "mistral-large-latest",
    temperature: 0,
});

const supervisorChain = formattedPrompt
    .pipe(llm.bindTools(
        [routingTool],
        {
            tool_choice: "any",
        },
    ))
    // select the first one
    .pipe((x) => (x.tool_calls[0].args));

Error: API key MISTRAL_API_KEY is missing for MistralAI, but it is required.

In [ ]:
// TEST du choix du superviseur
import { HumanMessage } from "npm:@langchain/core/messages";

await supervisorChain.invoke({
    messages: [
        new HumanMessage({
            content: "Hoow is 2+5?",
        }),
    ],
});

{ next: "number_additionner" }

In [51]:
// création des agents et de leurs nodes
import { RunnableConfig } from "npm:@langchain/core/runnables";
import { createReactAgent } from "npm:@langchain/langgraph/prebuilt";
import { SystemMessage } from "npm:@langchain/core/messages";

// Recall llm was defined as ChatOpenAI above
// It could be any other language model
const researcherAgent = createReactAgent({
    llm,
    tools: [tavilyTool],
    stateModifier: new SystemMessage("You are a web researcher. You may use the Tavily search engine to search the web for" +
        " important information, so the Chart Generator in your team can make useful plots.")
})

const researcherNode = async (
    state: typeof AgentState.State,
    config?: RunnableConfig,
) => {
    const result = await researcherAgent.invoke(state, config);
    const lastMessage = result.messages[result.messages.length - 1];
    return {
        messages: [
            new HumanMessage({ content: lastMessage.content, name: "Researcher" }),
        ],
    };
};

const mathematicAgent = createReactAgent({
    llm,
    tools: [additionerTool],
    stateModifier: new SystemMessage("You excel at number computing. Use the diffeent tool for the differents operations.")
})

const mathematicNode = async (
    state: typeof AgentState.State,
    config?: RunnableConfig,
) => {
    const result = await mathematicAgent.invoke(state, config);
    const lastMessage = result.messages[result.messages.length - 1];
    return {
        messages: [
            new HumanMessage({ content: lastMessage.content, name: "Mathematician" }),
        ],
    };
};

In [53]:
import { START, StateGraph } from "npm:@langchain/langgraph";

// 1. Create the graph
const workflow = new StateGraph(AgentState)
    // 2. Add the nodes; these will do the work
    .addNode("researcher", researcherNode)
    .addNode("number_additionner", mathematicNode)
    .addNode("supervisor", supervisorChain);
// 3. Define the edges. We will define both regular and conditional ones
// After a worker completes, report to supervisor
members.forEach((member) => {
    workflow.addEdge(member, "supervisor");
});

workflow.addConditionalEdges(
    "supervisor",
    (x: typeof AgentState.State) => x.next,
);

workflow.addEdge(START, "supervisor");

const graph = workflow.compile();

StateGraph {
  nodes: {
    researcher: {
      runnable: RunnableCallable {
        lc_serializable: false,
        lc_kwargs: {},
        lc_runnable: true,
        name: "researcher",
        lc_namespace: [ "langgraph" ],
        func: [AsyncFunction: researcherNode],
        tags: undefined,
        config: undefined,
        trace: false,
        recurse: true
      },
      retryPolicy: undefined,
      metadata: undefined,
      input: {
        messages: BinaryOperatorAggregate {
          ValueType: undefined,
          UpdateType: undefined,
          lg_is_channel: true,
          lc_graph_name: "BinaryOperatorAggregate",
          value: [],
          operator: [Function: reducer],
          initialValueFactory: [Function: default]
        },
        next: BinaryOperatorAggregate {
          ValueType: undefined,
          UpdateType: undefined,
          lg_is_channel: true,
          lc_graph_name: "BinaryOperatorAggregate",
          value: "__end__",
          operator

In [54]:
let streamResults = graph.stream(
    {
        messages: [
            new HumanMessage({
                content: "What were the 3 most popular tv shows in 2023?",
            }),
        ],
    },
    { recursionLimit: 100 },
);

for await (const output of await streamResults) {
    if (!output?.__end__) {
        console.log(output);
        console.log("----");
    }
}

{ supervisor: { next: "researcher" } }
----
{
  researcher: {
    messages: [
      HumanMessage {
        "content": "The 3 most popular tv shows in 2023 were:\n\n1. NFL Sunday Night Football (NBC)\n2. NFL Thursday Night Football (Amazon Prime)\n3. NFL Monday Night Football on ESPN (ESPN)",
        "name": "Researcher",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ]
  }
}
----
{ supervisor: { next: "__end__" } }
----
